In [1]:
# Import statements
import pandas as pd
import sklearn.model_selection
import sklearn.compose # column transformers
import sklearn.preprocessing
from sklearn.pipeline import Pipeline

In [3]:
# read in data
data = pd.read_csv("dataset_diabetes/diabetic_data.csv")
## should make patients unique by id?
data.drop_duplicates(["patient_nbr"])

In [6]:
X, y = data.loc[:, :"readmitted"], data.loc[:,"readmitted"]

In [10]:
Xtrain, ytrain, Xtest, ytest = sklearn.model_selection.train_test_split(X, y, \
                                                    test_size = 0.20, train_size = 0.80, shuffle = True)

In [23]:
Xtrain = Xtrain[Xtrain.columns[~Xtrain.columns.str.contains("_id")]] #drop all id columns
Xtrain.drop(["patient_nbr"], axis=1)

,race,gender,age,weight,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
20959,Caucasian,Male,[70-80),?,4,?,Orthopedics-Reconstructive,22,1,12,...,No,Up,No,No,No,No,No,Ch,Yes,>30
60621,Caucasian,Male,[70-80),?,1,SP,Family/GeneralPractice,24,5,14,...,No,No,No,No,No,No,No,Ch,Yes,NO
24808,AfricanAmerican,Female,[70-80),?,6,?,Family/GeneralPractice,46,0,13,...,No,No,No,No,No,No,No,No,No,NO
41974,Caucasian,Female,[90-100),?,4,?,?,55,2,12,...,No,No,No,No,No,No,No,No,Yes,NO
37643,Other,Male,[30-40),?,2,SP,Family/GeneralPractice,8,0,4,...,No,No,No,No,No,No,No,No,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41104,Caucasian,Female,[80-90),?,8,MC,Emergency/Trauma,20,2,13,...,No,Steady,No,No,No,No,No,No,Yes,NO
99116,Caucasian,Female,[50-60),?,6,MD,Emergency/Trauma,63,0,33,...,No,Down,No,No,No,No,No,Ch,Yes,>30
35430,Caucasian,Female,[70-80),?,8,MC,Surgery-Cardiovascular/Thoracic,49,6,56,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
60445,Hispanic,Female,[60-70),?,9,MD,?,86,3,30,...,No,Steady,No,No,No,No,No,No,Yes,>30


In [21]:
def binary_converter():
    
binary_transformer = sklearn.preprocessing.FunctionTransformer(binary_converter)

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')


In [ ]:
## full preprocessing pipeline from the training data
pipeline = Pipeline()

In [ ]:
## test it out on the cv data

In [ ]:
## and export it to a file for use on the test data